In [1]:
# Import necessary libraries
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, dash_table
import dash_leaflet as dl
import pandas as pd
import plotly.express as px  # Import plotly.express
from crud_module import AnimalShelter
import base64

# Initialize the Dash app
app = JupyterDash(__name__)

# MongoDB connection variables
USER = 'aacuser'
PASS = 'SNHU1234'
HOST = 'nv-desktop-services.apporto.com'
PORT = 31992
DB = 'AAC'
COL = 'animals'

# Instantiate the CRUD class
shelter = AnimalShelter(USER, PASS, HOST, PORT, DB, COL)

# Retrieve initial data from MongoDB
data = shelter.read({})
df = pd.DataFrame(data)

# Remove the '_id' column to avoid issues with the data table
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Encode the Grazioso Salvare logo
image_filename = 'Grazioso Salvare Logo.png'  # Replace with the correct path to your image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Define the layout of the app
app.layout = html.Div([
    html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'width': '200px'}), href='Documents/Grazioso Salvare Logo.png'),  # Add the Grazioso Salvare logo with a URL link
    html.H1('Grazioso Salvare Dashboard', style={'textAlign': 'center'}),
    html.H3('By Nathan Wilson', style={'textAlign': 'center'}),
    html.Hr(),
    
    # Interactive options for filtering data
    dcc.RadioItems(
        id='rescue-type-radio',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
            {'label': 'All', 'value': 'All'}
        ],
        value='All',
        labelStyle={'display': 'inline-block'}
    ),
    html.Br(),
    
    # Data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        style_table={'height': '300px', 'overflowY': 'auto'},
        row_selectable='single',
        selected_rows=[0],
        page_size=10,
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        }
    ),
    html.Br(),
    html.Hr(),
    
    # Layout for charts
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ]),
    
    # Unique Identifier
    html.H5("Your Unique Identifier: Nathan Wilson", style={'textAlign': 'center'}),
])

# Helper function to construct query based on rescue type
def construct_query(rescue_type):
    if rescue_type == 'Water Rescue':
        return {'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}}
    elif rescue_type == 'Mountain or Wilderness Rescue':
        return {'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}}
    elif rescue_type == 'Disaster or Individual Tracking':
        return {'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}}
    else:
        return {}

# Callback to update the data table based on the selected rescue type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-type-radio', 'value')]
)
def update_table(rescue_type):
    query = construct_query(rescue_type)
    data = shelter.read(query)
    df = pd.DataFrame(data)
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')

# Callback to update the map based on the selected row in the data table
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, selected_rows):
    if viewData is None or len(viewData) == 0:
        return ["No data available"]

    dff = pd.DataFrame.from_dict(viewData)
    if selected_rows is None or len(selected_rows) == 0:
        row = 0
    else:
        row = selected_rows[0]

    try:
        lat, lon = dff.iloc[row, [13, 14]]
    except IndexError:
        return ["No valid coordinates available"]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

# Callback to update the pie chart based on the selected rescue type
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_pie_chart(viewData):
    if viewData is None or len(viewData) == 0:
        return ["No data available"]

    dff = pd.DataFrame.from_dict(viewData)
    breed_counts = dff['breed'].value_counts()
    fig = px.pie(values=breed_counts.values, names=breed_counts.index, title='Breed Distribution')
    return [dcc.Graph(figure=fig)]

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


Dash app running on http://127.0.0.1:30814/
